<a href="https://colab.research.google.com/github/erivetna87/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/Eric_Rivetna_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import pprint as pp
from pandas.util.testing import assert_frame_equal
from scipy.stats import ttest_1samp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('seaborn-white')
pd.set_option('precision', 0)

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

df = pd.read_csv(url)

df.head(1)

cols = ['Class Name: 2 (democrat, republican)',
        'handicapped-infants: 2 (y,n)',
        'water-project-cost-sharing: 2 (y,n)',
        'adoption-of-the-budget-resolution: 2 (y,n)',
        'physician-fee-freeze: 2 (y,n)',
        'el-salvador-aid: 2 (y,n)',
        'religious-groups-in-schools: 2 (y,n)',
        'anti-satellite-test-ban: 2 (y,n)',
        'aid-to-nicaraguan-contras: 2 (y,n)',
        'mx-missile: 2 (y,n)',
        'immigration: 2 (y,n)',
        'synfuels-corporation-cutback: 2 (y,n)',
        'education-spending: 2 (y,n)',
        'superfund-right-to-sue: 2 (y,n)',
        'crime: 2 (y,n)',
        'duty-free-exports: 2 (y,n)',
        'export-administration-act-south-africa: 2 (y,n)']

df.columns = cols
df.rename(columns = lambda x: x.replace(': 2 (y,n)','')[0:],inplace = True)
df.rename(columns = lambda x: x.replace('Class Name: 2 (democrat, republican)',
                                        'Political Party')[0:],inplace = True)
df.columns = map(str.title, df.columns)

#to call column list in future cells as needed.
cols = df.columns.get_values().tolist()
# df.head(1)






In [850]:
print('Head of 1980s congressional voting data:')
print('\n')
print(df.head(5))
print('\n')
print('\n')
print('Tail of 1980s congressional voting data:')
print('\n')
print(df.tail(5))




Head of 1980s congressional voting data:


  Political Party Handicapped-Infants Water-Project-Cost-Sharing Adoption-Of-The-Budget-Resolution Physician-Fee-Freeze El-Salvador-Aid Religious-Groups-In-Schools Anti-Satellite-Test-Ban Aid-To-Nicaraguan-Contras Mx-Missile Immigration Synfuels-Corporation-Cutback Education-Spending Superfund-Right-To-Sue Crime Duty-Free-Exports Export-Administration-Act-South-Africa
0      republican                   n                          y                                 n                    y               y                           y                       n                         n          n           n                            n                  y                      y     y                 n                                      ?
1        democrat                   ?                          y                                 y                    ?               y                           y                       n                         n  

In [851]:
#Checking for null values
df.isnull().sum()

Political Party                           0
Handicapped-Infants                       0
Water-Project-Cost-Sharing                0
Adoption-Of-The-Budget-Resolution         0
Physician-Fee-Freeze                      0
El-Salvador-Aid                           0
Religious-Groups-In-Schools               0
Anti-Satellite-Test-Ban                   0
Aid-To-Nicaraguan-Contras                 0
Mx-Missile                                0
Immigration                               0
Synfuels-Corporation-Cutback              0
Education-Spending                        0
Superfund-Right-To-Sue                    0
Crime                                     0
Duty-Free-Exports                         0
Export-Administration-Act-South-Africa    0
dtype: int64

In [852]:
#Looking at all Unique Values in each df series
[df[str(i)].unique() for i in cols]

[array(['republican', 'democrat'], dtype=object),
 array(['n', '?', 'y'], dtype=object),
 array(['y', 'n', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['y', '?', 'n'], dtype=object),
 array(['y', '?', 'n'], dtype=object),
 array(['y', 'n', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['y', 'n', '?'], dtype=object),
 array(['y', '?', 'n'], dtype=object),
 array(['y', 'n', '?'], dtype=object),
 array(['n', 'y', '?'], dtype=object),
 array(['?', 'n', 'y'], dtype=object)]

In [0]:
df.replace({'?':np.NaN,'y': 1,'n':0},inplace=True)

In [854]:
#Second check for null values - NaN will be dropped in statistical analysis
df.isnull().sum()

Political Party                             0
Handicapped-Infants                        12
Water-Project-Cost-Sharing                 48
Adoption-Of-The-Budget-Resolution          11
Physician-Fee-Freeze                       11
El-Salvador-Aid                            15
Religious-Groups-In-Schools                11
Anti-Satellite-Test-Ban                    14
Aid-To-Nicaraguan-Contras                  15
Mx-Missile                                 22
Immigration                                 7
Synfuels-Corporation-Cutback               20
Education-Spending                         31
Superfund-Right-To-Sue                     25
Crime                                      17
Duty-Free-Exports                          28
Export-Administration-Act-South-Africa    104
dtype: int64

In [0]:
#Separating party lines in DataFrames

rep = df[df['Political Party'] == 'republican']

In [0]:
dem = df[df['Political Party'] == 'democrat']

In [857]:
df_assert = rep + dem

if df.shape == df_assert.shape:
  print(df['Political Party'].value_counts())
  print('Combined DataFrames are Equal to Original DataFrame')
else:
  print('DataFrames not Equal')


democrat      267
republican    167
Name: Political Party, dtype: int64
Combined DataFrames are Equal to Original DataFrame


# Political Party T-test


In [858]:
rep['Handicapped-Infants'].mean()

0.18902439024390244

In [859]:
# Null Hypothesis: Republican support is evenly divided
# Alternative: Republican support is not evenly divided.
pp.pprint([str(i) + ' 1 samp t-test: ' + str(ttest_1samp(rep[str(i)], .5, nan_policy='omit')) for i in cols[1:]])

  

['Handicapped-Infants 1 samp t-test: '
 'Ttest_1sampResult(statistic=-10.14045193578877, '
 'pvalue=4.848137183614312e-19)',
 'Water-Project-Cost-Sharing 1 samp t-test: '
 'Ttest_1sampResult(statistic=0.08219949365267874, pvalue=0.9346006601350887)',
 'Adoption-Of-The-Budget-Resolution 1 samp t-test: '
 'Ttest_1sampResult(statistic=-13.597328074086908, '
 'pvalue=1.400603202875075e-28)',
 'Physician-Fee-Freeze 1 samp t-test: '
 'Ttest_1sampResult(statistic=56.74286815468313, '
 'pvalue=2.907577352774837e-109)',
 'El-Salvador-Aid 1 samp t-test: '
 'Ttest_1sampResult(statistic=26.743499090241418, pvalue=1.6943192044621e-61)',
 'Religious-Groups-In-Schools 1 samp t-test: '
 'Ttest_1sampResult(statistic=16.722758024023662, '
 'pvalue=2.8338200332433206e-37)',
 'Anti-Satellite-Test-Ban 1 samp t-test: '
 'Ttest_1sampResult(statistic=-7.610196474080465, '
 'pvalue=2.220472808251682e-12)',
 'Aid-To-Nicaraguan-Contras 1 samp t-test: '
 'Ttest_1sampResult(statistic=-11.944473807809807, '
 'pvalu

In [860]:
# Null Hypothesis: There is 0 support for bills among Republicans in the House
# Alternative: There is non-0 support (some support) for some bills
pp.pprint([str(i) + ' 1 samp t-test: ' + str(ttest_1samp(rep[str(i)], 0, nan_policy='omit')) for i in cols[1:]])

['Handicapped-Infants 1 samp t-test: '
 'Ttest_1sampResult(statistic=6.163804117832389, pvalue=5.375393477765494e-09)',
 'Water-Project-Cost-Sharing 1 samp t-test: '
 'Ttest_1sampResult(statistic=12.16552506059644, '
 'pvalue=5.9376882505675456e-24)',
 'Adoption-Of-The-Budget-Resolution 1 samp t-test: '
 'Ttest_1sampResult(statistic=5.027583489578352, '
 'pvalue=1.3042608973305829e-06)',
 'Physician-Fee-Freeze 1 samp t-test: '
 'Ttest_1sampResult(statistic=114.90430801323333, '
 'pvalue=6.637172822355131e-158)',
 'El-Salvador-Aid 1 samp t-test: '
 'Ttest_1sampResult(statistic=56.378187271319746, '
 'pvalue=7.906153477470814e-109)',
 'Religious-Groups-In-Schools 1 samp t-test: '
 'Ttest_1sampResult(statistic=37.78577385580919, pvalue=7.590936296107066e-83)',
 'Anti-Satellite-Test-Ban 1 samp t-test: '
 'Ttest_1sampResult(statistic=7.151750903352727, '
 'pvalue=2.8958659572948416e-11)',
 'Aid-To-Nicaraguan-Contras 1 samp t-test: '
 'Ttest_1sampResult(statistic=5.308655025693248, '
 'pvalu